# boto3のテスト

In [ ]:
import boto3

In [ ]:
s3 = boto3.client("s3", region_name="ap-notrheast-1")

In [ ]:
client = boto3.client("dynamodb", region_name="ap-northeast-1")
# resp = client.create_table(
#     TableName="test",
#     KeySchema=[
#         {
#             "AttributeName": "code",
#             "KeyType": "HASH"
#         },
#         {
#             "AttributeName": "date",
#             "KeyType": "RANGE"  
#         }
#     ],
#     AttributeDefinitions=[
#         {
#             "AttributeName": "code",
#             "AttributeType": "S"
#         },
#         {
#             "AttributeName": "date",
#             "AttributeType": "S"
#         }
#     ],
#     ProvisionedThroughput={
#         "ReadCapacityUnits": 5,
#         "WriteCapacityUnits": 5
#     }
# )

In [ ]:
client.list_tables()

In [ ]:
dynamodb = boto3.resource("dynamodb", region_name="ap-northeast-1")

In [ ]:
table_name = "stock_data"

is_table_exist = len([
        tbl for tbl in dynamodb.tables.all() if tbl.name == table_name
    ]) > 0  

In [ ]:
if not is_table_exist:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                "AttributeName": "code",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "date",
                "KeyType": "RANGE"
            }
        ],
        AttributeDefinitions=[
            {
                "AttributeName": "code",
                "AttributeType": "S"
            },
            {
                "AttributeName": "date",
                "AttributeType": "S"
            }
        ],
        ProvisionedThroughput={
            "ReadCapacityUnits": 5,
            "WriteCapacityUnits": 5
        }
    )
    table.wait_until_exists()
else:
    table = dynamodb.Table(table_name)

print(table.item_count)

In [ ]:
table.put_item(
    Item={
        "code": "foo",
        "date": "20221124",
        "timestamps": [1, 2, 3, 4, 5, 6,]
    }
)


In [ ]:
response = table.get_item(Key={
    "code": "foo",
    "date": "20221124"
})

item = response["Item"]
print(item["timestamps"])

In [ ]:
## yahoo financeからデータ取得 -> dynamodbに保存
import decimal
from datetime import datetime
import stock

code = "META"

data = stock.scraping.yahoo_finance.get_stock_time_series(code, "1m", "1d", include_pre_post=True)
date = datetime.fromtimestamp(data.timestamp[-1]).strftime("%Y%m%d")
    
def to_decimal(arr):
    return [decimal.Decimal(str(val)) for val in arr]

# table.put_item(Item={
#     "code": code,
#     "date": date,
#     "timestamp": data.timestamp,
#     "open": to_decimal(data.open),
#     "high": to_decimal(data.high),
#     "low": to_decimal(data.low),
#     "close": to_decimal(data.close),
#     "volume": to_decimal(data.volume),
# })

In [ ]:
import json

with open("./tmp.json", "w") as f:
    json.dump(data, f)